In [17]:
import pandas as pd
import os
import json
from functools import singledispatch

In [18]:
STATE_NAMES = {
  'AP': 'Andhra Pradesh',
  'AR': 'Arunachal Pradesh',
  'AS': 'Assam',
  'BR': 'Bihar',
  'CT': 'Chhattisgarh',
  'GA': 'Goa',
  'GJ': 'Gujarat',
  'HR': 'Haryana',
  'HP': 'Himachal Pradesh',
  'JH': 'Jharkhand',
  'KA': 'Karnataka',
  'KL': 'Kerala',
  'MP': 'Madhya Pradesh',
  'MH': 'Maharashtra',
  'MN': 'Manipur',
  'ML': 'Meghalaya',
  'MZ': 'Mizoram',
  'NL': 'Nagaland',
  'OR': 'Odisha',
  'PB': 'Punjab',
  'RJ': 'Rajasthan',
  'SK': 'Sikkim',
  'TN': 'Tamil Nadu',
  'TG': 'Telangana',
  'TR': 'Tripura',
  'UT': 'Uttarakhand',
  'UP': 'Uttar Pradesh',
  'WB': 'West Bengal',
  'AN': 'Andaman and Nicobar Islands',
  'CH': 'Chandigarh',
  'DN': 'Dadra and Nagar Haveli and Daman and Diu',
  'DL': 'Delhi',
  'JK': 'Jammu and Kashmir',
  'LA': 'Ladakh',
  'LD': 'Lakshadweep',
  'PY': 'Puducherry',
#   'TT': 'India',
 # [UNASSIGNED_STATE_CODE]: 'Unassigned',
}

In [19]:
data_min_json={}
for k,v in STATE_NAMES.items():
    data_min_json[k]={"districts":{},"delta":{},"delta7":{},"delta21_14":{},"meta":{},"total":{}}

In [20]:
def number_generation(df,col_name,col_value,field):
    value=df.loc[df[col_name]==col_value,field]
    value.reset_index(inplace=True,drop=True)
    if value.isna().any():
        value = "null"
    try:
        value = int(value[0])
    except:
        value = value[0]
    return value

In [22]:
for k,v in data_min_json.items():
    df=pd.read_csv("../RAWCSV/2021-10-30/"+k+"_final.csv")
    # df = pd.read_csv('datamin.csv',header=[1])
    for district in df["District"]:
        try:
            district_dict={district:{"delta":{"comfirmed":number_generation(df,"District",district,"deltaConfirmedForDistrict"),
                                              "recoverd":number_generation(df,"District",district,"deltaRecoveredForDistrict"),
                                              "deceased":number_generation(df,"District",district,"deltaDeceasedForDistrict"),
                                              # "vaccinated":number_generation(df,"District",district,"deltaVaccinatedForDistrict")
                                             }}}
        except KeyError as e:
            print(district,e)
            pass
        else:
            data_min_json[k]["districts"].update(district_dict)
        
    data_min_json[k]["delta"]["confirmed"]=number_generation(df,"State/UTCode",k,'deltaConfirmedForState')
    data_min_json[k]["delta"]["deceased"]=number_generation(df,"State/UTCode",k,'deltaDeceasedForState')
    data_min_json[k]["delta"]["recovered"]=number_generation(df,"State/UTCode",k,'deltaRecoveredForState')
    data_min_json[k]["delta"]["vaccinated"]=number_generation(df,"State/UTCode",k,'deltaVaccinatedForState')
    data_min_json[k]["delta"]["tested"]=number_generation(df,"State/UTCode",k,'deltaTestedForState')
    
    data_min_json[k]["delta7"]["confirmed"]=number_generation(df,"State/UTCode",k,'7DmaConfirmedForState')
    data_min_json[k]["delta7"]["deceased"]=number_generation(df,"State/UTCode",k,'7DmaDeceasedForState')
    data_min_json[k]["delta7"]["recovered"]=number_generation(df,"State/UTCode",k,'7DmaRecoveredForState')
    data_min_json[k]["delta7"]["vaccinated"]=number_generation(df,"State/UTCode",k,'7DmaVaccinatedForState')
    data_min_json[k]["delta7"]["tested"]=number_generation(df,"State/UTCode",k,'7DmaTestedForState')
    data_min_json[k]["delta21_14"]={"confirmed":number_generation(df,"State/UTCode",k,'delta21_14confirmedForState')}
    
    data_min_json[k]["meta"]={"Date":number_generation(df,"State/UTCode",k,'Date'),
                             "last_updated":number_generation(df,"State/UTCode",k,'last_updated'),
                              "population":number_generation(df,"State/UTCode",k,'statePopulation'),
                              "tested":{"date":number_generation(df,"State/UTCode",k,'last_updated'),
                                        "source":number_generation(df,"State/UTCode",k,'tested_source_state')
                              }
                             }
    data_min_json[k]["total"] = {"confirmed":number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState'),
                                "deceased":number_generation(df,"State/UTCode",k,'cumulativeDeceasedNumberForState'),
                                "recovered":number_generation(df,"State/UTCode",k,'cumulativeRecoveredNumberForState'),
                                "tested":number_generation(df,"State/UTCode",k,'cumulativeTestedNumberForState'),
                                "vaccinated1":number_generation(df,"State/UTCode",k,'cumulativeVaccinated1NumberForState'),
                                "vaccinated2":number_generation(df,"State/UTCode",k,'cumulativeVaccinated2NumberForState')
                                }

nan 0


In [23]:
@singledispatch
def remove_null_bool(ob):
    return ob

@remove_null_bool.register(list)
def _process_list(ob):
    return [remove_null_bool(v) for v in ob]

@remove_null_bool.register(dict)
def _process_list(ob):
    return {k: remove_null_bool(v) for k, v in ob.items()
            if v is not None and v is not 0}

with open('data.min_swi.json', 'w') as json_file:
    json.dump(remove_null_bool(data_min_json), json_file)

In [12]:
import requests, json

In [24]:
url = requests.get("https://data.covid19india.org/v4/min/data.min.json")
text = url.text

In [14]:
C19I_data = json.loads(text)

In [35]:
compare_json_data(C19I_data['RJ'],remove_null_bool(data_min_json))

False

In [36]:
print('Compare JSON result is: {0}'.format(
		compare_json_data(C19I_data['RJ'],remove_null_bool(data_min_json))
	))

Compare JSON result is: False


In [21]:
!pip install jsondiff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for jsondiff, since package 'wheel' is not installed.
    Running setup.py install for jsondiff: started
    Running setup.py install for jsondiff: finished with status 'done'


In [11]:
import jsondiff

In [25]:
res = jsondiff.diff(C19I_data,remove_null_bool(data_min_json))

In [27]:
remove_null_bool(data_min_json)

{'AP': {'districts': {'Anantapur': {'delta': {}},
   'Chittoor': {'delta': {}},
   'East Godavari': {'delta': {}},
   'Guntur': {'delta': {}},
   'Y.S.R. Kadapa': {'delta': {}},
   'Krishna': {'delta': {}},
   'Kurnool': {'delta': {}},
   'S.P.S. Nellore': {'delta': {}},
   'Prakasam': {'delta': {}},
   'Srikakulam': {'delta': {}},
   'Visakhapatnam': {'delta': {}},
   'Vizianagaram': {'delta': {}}},
  'delta': {'confirmed': 'n',
   'deceased': 'n',
   'recovered': 'n',
   'vaccinated': 'n'},
  'delta7': {'confirmed': 'n',
   'deceased': 'n',
   'recovered': 'n',
   'vaccinated': 51832006,
   'tested': 'n'},
  'delta21_14': {},
  'meta': {'Date': '2021-10-30',
   'last_updated': 'n',
   'population': 52221000,
   'tested': {'date': 'n', 'source': 'n'}},
  'total': {'confirmed': 'n',
   'deceased': 'n',
   'recovered': 'n',
   'tested': 'n',
   'vaccinated1': 32955532,
   'vaccinated2': 20350068}},
 'AR': {'districts': {'Anjaw': {'delta': {}},
   'Changlang': {'delta': {}},
   'East Kam

In [40]:
42374177 + 19518213

61892390